In [1]:
! pip install glaredb --upgrade

In [2]:
import glaredb

In [3]:
con = glaredb.connect()

In [4]:
! ls

COPY_TO_local_files_with_GlareDB.ipynb suppliers.csv
northwind.db


In [5]:
con.sql(
    """
        COPY (select 1) TO 'copy.csv'
    """
)

Noop

In [6]:
! ls

COPY_TO_local_files_with_GlareDB.ipynb northwind.db
copy.csv                               suppliers.csv


In [7]:
con.sql(
    """
        SELECT * FROM read_sqlite('./northwind.db', 'Products') LIMIT 5
    """
    ).to_pandas()

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1


In [8]:
con.sql(
    """
        SELECT * FROM './suppliers.csv' LIMIT 5
    """
).to_pandas()

,id,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,HomePage
0,1,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,British Isles,EC1 4SD,UK,(171) 555-2222,None,None
1,2,New Orleans Cajun Delights,Shelley Burke,Order Administrator,P.O. Box 78934,New Orleans,North America,70117,USA,(100) 555-4822,None,#CAJUN.HTM#
2,3,Grandma Kelly's Homestead,Regina Murphy,Sales Representative,707 Oxford Rd.,Ann Arbor,North America,48104,USA,(313) 555-5735,(313) 555-3349,None
3,4,Tokyo Traders,Yoshi Nagase,Marketing Manager,9-8 Sekimai\nMusashino-shi,Tokyo,Eastern Asia,100,Japan,(03) 3555-5011,None,None
4,5,Cooperativa de Quesos 'Las Cabras',Antonio del Valle Saavedra,Export Administrator,Calle del Rosal 4,Oviedo,Southern Europe,33007,Spain,(98) 598 76 54,None,None


In [9]:
con.sql(
    """
        SELECT * 
        FROM read_sqlite('./northwind.db', 'Products') p
        JOIN read_csv('./suppliers.csv') s
        ON s."id" = p."SupplierID"
    """
).to_pandas()

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued,...,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,HomePage
0,1,Chai,1,1,10 boxes x 20 bags,18,39,0,10,0,...,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,British Isles,EC1 4SD,UK,(171) 555-2222,None,None
1,2,Chang,1,1,24 - 12 oz bottles,19,17,40,25,0,...,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,British Isles,EC1 4SD,UK,(171) 555-2222,None,None
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10,13,70,25,0,...,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,British Isles,EC1 4SD,UK,(171) 555-2222,None,None
3,33,Geitost,15,4,500 g,2.5,112,0,20,0,...,Beate Vileid,Marketing Manager,Hatlevegen 5,Sandvika,Scandinavia,1320,Norway,(0)2-953010,None,None
4,69,Gudbrandsdalsost,15,4,10 kg pkg.,36,26,0,15,0,...,Beate Vileid,Marketing Manager,Hatlevegen 5,Sandvika,Scandinavia,1320,Norway,(0)2-953010,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,26,Gumbär Gummibärchen,11,3,100 - 250 g bags,31.23,15,0,0,0,...,Petra Winkler,Sales Manager,Tiergartenstraße 5,Berlin,Western Europe,10785,Germany,(010) 9984510,None,None
73,27,Schoggi Schokolade,11,3,100 - 100 g pieces,43.9,49,0,30,0,...,Petra Winkler,Sales Manager,Tiergartenstraße 5,Berlin,Western Europe,10785,Germany,(010) 9984510,None,None
74,42,Singaporean Hokkien Fried Mee,20,5,32 - 1 kg pkgs.,14,26,0,0,1,...,Chandra Leka,Owner,"471 Serangoon Loop, Suite #402",Singapore,South-East Asia,0512,Singapore,555-8787,None,None
75,43,Ipoh Coffee,20,1,16 - 500 g tins,46,17,10,25,0,...,Chandra Leka,Owner,"471 Serangoon Loop, Suite #402",Singapore,South-East Asia,0512,Singapore,555-8787,None,None


In [10]:
con.sql("""
    COPY (
    SELECT * FROM read_sqlite('./northwind.db', 'Products') p
    JOIN read_csv('./suppliers.csv') s
    ON s."id" = p."SupplierID"
    ) TO './joined_data.csv'
""").to_pandas()

""


In [11]:
!ls

COPY_TO_local_files_with_GlareDB.ipynb northwind.db
copy.csv                               suppliers.csv
joined_data.csv


In [12]:
con.sql("""
    SELECT * FROM './joined_data.csv'
""").to_pandas()

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued,...,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,HomePage
0,31,Gorgonzola Telino,14,4,12 - 100 g pkgs,12.50,0,70,20,0,...,Elio Rossi,Sales Representative,"Viale Dante, 75",Ravenna,Southern Europe,48100,Italy,(0544) 60323,(0544) 60603,#FORMAGGI.HTM#
1,32,Mascarpone Fabioli,14,4,24 - 200 g pkgs.,32.00,9,40,25,0,...,Elio Rossi,Sales Representative,"Viale Dante, 75",Ravenna,Southern Europe,48100,Italy,(0544) 60323,(0544) 60603,#FORMAGGI.HTM#
2,72,Mozzarella di Giovanni,14,4,24 - 200 g pkgs.,34.80,14,0,0,0,...,Elio Rossi,Sales Representative,"Viale Dante, 75",Ravenna,Southern Europe,48100,Italy,(0544) 60323,(0544) 60603,#FORMAGGI.HTM#
3,40,Boston Crab Meat,19,8,24 - 4 oz tins,18.40,123,0,30,0,...,Robb Merchant,Wholesale Account Agent,Order Processing Dept.\n2100 Paul Revere Blvd.,Boston,North America,02134,USA,(617) 555-3267,(617) 555-3389,None
4,41,Jack's New England Clam Chowder,19,8,12 - 12 oz cans,9.65,85,0,10,0,...,Robb Merchant,Wholesale Account Agent,Order Processing Dept.\n2100 Paul Revere Blvd.,Boston,North America,02134,USA,(617) 555-3267,(617) 555-3389,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,26,Gumbär Gummibärchen,11,3,100 - 250 g bags,31.23,15,0,0,0,...,Petra Winkler,Sales Manager,Tiergartenstraße 5,Berlin,Western Europe,10785,Germany,(010) 9984510,None,None
73,27,Schoggi Schokolade,11,3,100 - 100 g pieces,43.90,49,0,30,0,...,Petra Winkler,Sales Manager,Tiergartenstraße 5,Berlin,Western Europe,10785,Germany,(010) 9984510,None,None
74,42,Singaporean Hokkien Fried Mee,20,5,32 - 1 kg pkgs.,14.00,26,0,0,1,...,Chandra Leka,Owner,"471 Serangoon Loop, Suite #402",Singapore,South-East Asia,0512,Singapore,555-8787,None,None
75,43,Ipoh Coffee,20,1,16 - 500 g tins,46.00,17,10,25,0,...,Chandra Leka,Owner,"471 Serangoon Loop, Suite #402",Singapore,South-East Asia,0512,Singapore,555-8787,None,None


In [ ]:
! ls

In [ ]:
con.sql(
    """
        SELECT * FROM './joined_data.csv'
    """
).to_pandas()